In [ ]:
print('Setup complete.')

# AI UI Builder Demo

## Learning Objectives
- Generate interactive user interfaces using AI assistance
- Build Streamlit dashboards from specifications
- Create responsive web components with AI guidance
- Design user-centered interface workflows

## The Challenge: Rapid UI Prototyping

This demo shows how to use AI to rapidly prototype and build interactive user interfaces. We'll create:
1. **Streamlit Dashboards** - Data visualization and interaction
2. **HTML/CSS Components** - Custom web interface elements
3. **Interactive Widgets** - User input and feedback systems
4. **Responsive Design** - Mobile-friendly layouts

In [ ]:
# Setup and imports
!pip install asksageclient pip_system_certs
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import time
import tiktoken
from pathlib import Path
from typing import Dict, List, Any

# Import our AskSage client
from asksageclient import AskSageClient

# Get API credentials from Google Colab secrets
from google.colab import userdata
api_key = userdata.get('ASKSAGE_API_KEY')
email = userdata.get('ASKSAGE_EMAIL')

# Initialize client and tokenizer
client = AskSageClient(api_key=api_key, email=email)
tokenizer = tiktoken.encoding_for_model("gpt-4")
print("AskSage client initialized successfully")
print("Ready to showcase AI capabilities...")

In [ ]:
# ================================
# 🔐 Cell 1 — Load secrets (Colab) + pricing + token utils
# ================================
import os, time, csv
from typing import Optional, Dict
import tiktoken

from google.colab import userdata

ASKSAGE_API_KEY = userdata.get("ASKSAGE_API_KEY")
ASKSAGE_BASE_URL = userdata.get("ASKSAGE_BASE_URL")
ASKSAGE_EMAIL = userdata.get("ASKSAGE_EMAIL")

assert ASKSAGE_API_KEY, "ASKSAGE_API_KEY not provided."
assert ASKSAGE_EMAIL, "ASKSAGE_EMAIL not provided."

print("✓ Secrets loaded")
print("  • EMAIL:", ASKSAGE_EMAIL)
print("  • BASE URL:", ASKSAGE_BASE_URL or "(default)")

# Pricing (USD per 1,000,000 tokens)
PRICES_PER_M = {
    "gpt-5": {"input_per_m": 1.25, "output_per_m": 10.00},
    "gpt-5-mini": {"input_per_m": 0.25, "output_per_m": 2.00},
}

# Tokenizer
enc = tiktoken.get_encoding("o200k_base")

def count_tokens(text: str) -> int:
    return len(enc.encode(text or ""))

def cost_usd(model: str, input_tokens: int, output_tokens: int) -> float:
    if model not in PRICES_PER_M:
        raise ValueError(f"Unknown model: {model}")
    r = PRICES_PER_M[model]
    return (input_tokens / 1_000_000) * r["input_per_m"] + (output_tokens / 1_000_000) * r["output_per_m"]

In [ ]:
# Setup and imports
!pip install asksageclient pip_system_certs
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import time
import tiktoken
from pathlib import Path
from typing import Dict, List, Any

# Import our AskSage client
from asksageclient import AskSageClient

# Get API credentials from Google Colab secrets
from google.colab import userdata
api_key = userdata.get('ASKSAGE_API_KEY')
email = userdata.get('ASKSAGE_EMAIL')

# Initialize client and tokenizer
client = AskSageClient(api_key=api_key, email=email)
tokenizer = tiktoken.encoding_for_model("gpt-4")
print("AskSage client initialized successfully")
print("Ready to showcase AI capabilities...")

In [ ]:
import os
from typing import Dict, List, Optional, Any
from dataclasses import dataclass

import openai
from rich.console import Console
from rich.panel import Panel
from rich.syntax import Syntax

console = Console()
print("🎨 AI UI Builder loading...")

## AI UI Generator

In [ ]:
class AIUIGenerator:
    """Generate user interfaces using AI assistance"""
    
    def __init__(self):
        self.setup_client()
        self.generated_uis = {}
    
    def setup_client(self):
        """Setup API client"""
        if os.getenv('OPENAI_API_KEY'):
            try:
                self.client = openai.OpenAI()
                self.has_api = True
                console.print("✅ OpenAI client configured")
            except Exception as e:
                self.has_api = False
                console.print(f"⚠️ Using mock responses: {e}")
        else:
            self.has_api = False
            console.print("💡 No API key found, using mock responses")
    
    def generate_streamlit_app(self, spec: str) -> str:
        """Generate Streamlit application from specification"""
        
        prompt = f"""Create a complete Streamlit application based on this specification:

{spec}

Requirements:
1. Complete, runnable Streamlit code
2. Proper layout with sidebar and main area
3. Interactive widgets for user input
4. Data visualization using plotly or matplotlib
5. Clean, professional styling
6. Error handling and validation

Return only the Python code for the Streamlit app."""
        
        if self.has_api:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=1500,
                temperature=0.2
            )
            return response.choices[0].message.content
        else:
            # Mock Streamlit app
            return '''import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np

st.set_page_config(page_title="Data Dashboard", layout="wide")

def main():
    st.title("🚀 AI-Generated Data Dashboard")
    st.markdown("Welcome to your custom analytics dashboard!")
    
    # Sidebar controls
    with st.sidebar:
        st.header("Configuration")
        data_points = st.slider("Data Points", 10, 1000, 100)
        chart_type = st.selectbox("Chart Type", ["Line", "Bar", "Scatter"])
        show_stats = st.checkbox("Show Statistics", True)
    
    # Generate sample data
    @st.cache_data
    def generate_data(n_points):
        dates = pd.date_range("2023-01-01", periods=n_points, freq="D")
        values = np.cumsum(np.random.randn(n_points)) + 100
        return pd.DataFrame({"Date": dates, "Value": values})
    
    df = generate_data(data_points)
    
    # Main content
    col1, col2 = st.columns([3, 1])
    
    with col1:
        st.subheader(f"{chart_type} Chart")
        if chart_type == "Line":
            fig = px.line(df, x="Date", y="Value")
        elif chart_type == "Bar":
            fig = px.bar(df.tail(20), x="Date", y="Value")
        else:
            fig = px.scatter(df, x="Date", y="Value")
        
        st.plotly_chart(fig, use_container_width=True)
    
    with col2:
        if show_stats:
            st.subheader("Statistics")
            st.metric("Mean", f"{df['Value'].mean():.2f}")
            st.metric("Std Dev", f"{df['Value'].std():.2f}")
            st.metric("Min", f"{df['Value'].min():.2f}")
            st.metric("Max", f"{df['Value'].max():.2f}")
    
    # Data table
    if st.checkbox("Show Raw Data"):
        st.subheader("Raw Data")
        st.dataframe(df, use_container_width=True)

if __name__ == "__main__":
    main()'''
    
    def generate_html_component(self, spec: str) -> str:
        """Generate HTML/CSS component"""
        
        prompt = f"""Create a responsive HTML/CSS component based on this specification:

{spec}

Requirements:
1. Clean, modern HTML5 structure
2. Responsive CSS with mobile-first design
3. Interactive elements with hover effects
4. Accessible design with proper ARIA labels
5. Professional styling with good UX

Return complete HTML with inline CSS."""
        
        if self.has_api:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=1200,
                temperature=0.2
            )
            return response.choices[0].message.content
        else:
            # Mock HTML component
            return '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI-Generated Component</title>
    <style>
        .dashboard-card {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            border-radius: 12px;
            padding: 24px;
            color: white;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            transition: transform 0.3s ease;
        }
        .dashboard-card:hover {
            transform: translateY(-5px);
        }
        .metric-value {
            font-size: 2.5rem;
            font-weight: bold;
            margin: 0;
        }
        .metric-label {
            font-size: 1rem;
            opacity: 0.8;
            margin-top: 8px;
        }
        @media (max-width: 768px) {
            .dashboard-card {
                padding: 16px;
            }
            .metric-value {
                font-size: 2rem;
            }
        }
    </style>
</head>
<body>
    <div class="dashboard-card">
        <h2 class="metric-value">1,247</h2>
        <p class="metric-label">Active Users</p>
    </div>
</body>
</html>'''

# Initialize UI generator
ui_generator = AIUIGenerator()
print("🎨 AI UI generator ready!")

## Demo: Generate Analytics Dashboard

In [ ]:
# Generate Streamlit dashboard
dashboard_spec = """
Create an interactive analytics dashboard with:
- File upload functionality for CSV/Excel files
- Data preview and summary statistics
- Multiple chart types (line, bar, scatter, histogram)
- Filtering and grouping controls
- Export functionality for charts and data
- Professional dark theme with company branding
"""

console.print("\n🎨 [bold blue]Generating Analytics Dashboard[/bold blue]")

streamlit_code = ui_generator.generate_streamlit_app(dashboard_spec)

# Display generated code
console.print("\n[green]✅ Streamlit dashboard generated![/green]")
streamlit_syntax = Syntax(streamlit_code[:1200] + "\n# ... rest of application ...", "python", theme="monokai", line_numbers=True)
console.print(Panel(streamlit_syntax, title="dashboard.py", border_style="green"))

# Save to file
with open("generated_dashboard.py", "w") as f:
    f.write(streamlit_code)

console.print("[cyan]💾 Saved to generated_dashboard.py[/cyan]")
print("\n🚀 Run with: streamlit run generated_dashboard.py")

## Demo: Generate HTML Component

In [ ]:
# Generate HTML component
component_spec = """
Create a metrics dashboard card component with:
- Large metric value display
- Trend indicator (up/down arrow with color)
- Comparison to previous period
- Hover effects and animations
- Responsive design for mobile/desktop
- Modern glassmorphism styling
"""

console.print("\n🎨 [bold blue]Generating HTML Component[/bold blue]")

html_code = ui_generator.generate_html_component(component_spec)

# Display generated HTML
console.print("\n[green]✅ HTML component generated![/green]")
html_syntax = Syntax(html_code[:800] + "\n<!-- ... rest of component ... -->", "html", theme="monokai", line_numbers=True)
console.print(Panel(html_syntax, title="metric_card.html", border_style="green"))

# Save to file
with open("metric_card.html", "w") as f:
    f.write(html_code)

console.print("[cyan]💾 Saved to metric_card.html[/cyan]")
print("\n🌐 Open metric_card.html in your browser to view")

## Key Takeaways: AI-Assisted UI Development

### 🎯 **UI Generation Best Practices**

1. **Clear Specifications**: Detailed requirements lead to better UI code
2. **Framework Selection**: Choose appropriate tools (Streamlit, HTML/CSS, React)
3. **Responsive Design**: Always consider mobile and desktop experiences
4. **Accessibility**: Include proper ARIA labels and semantic HTML
5. **Performance**: Optimize for fast loading and smooth interactions

### 🔧 **Streamlit Advantages**

- **Rapid Prototyping**: Quick data app development
- **Python Integration**: Seamless with data science workflows
- **Built-in Components**: Rich widget library
- **Deployment**: Easy sharing and hosting
- **Interactivity**: Real-time updates and user interaction

### 🚀 **Production Considerations**

- **User Testing**: Validate AI-generated UIs with real users
- **Brand Consistency**: Customize generated code to match brand guidelines
- **Performance Optimization**: Optimize loading times and responsiveness
- **Security**: Validate user inputs and sanitize data
- **Maintenance**: Plan for updates and feature additions

## Summary: Day 3 Demos Complete

🎉 **Congratulations!** You've completed all Day 3 demos:
- **Spec-to-Script**: Transform requirements into working code
- **Package Refactor**: Modernize legacy code architecture
- **Golden Tests**: Generate comprehensive test suites
- **AI UI Builder**: Create interactive interfaces rapidly

Ready for the Day 3 labs? Let's build practical AI-assisted development tools!